In [ ]:

# Fine-tune LLaMA-7B on MentalManip dataset using LoRA (PEFT)

# Step 1: Install dependencies
!pip install -q transformers datasets peft bitsandbytes accelerate

# Step 2: Load the tokenizer and model
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType
import torch

model_id = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# Step 3: Prepare PEFT config for LoRA
config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, config)

# Step 4: Load and preprocess the dataset
from datasets import load_dataset

dataset = load_dataset("csv", data_files={"train": "mentalmanip_train.csv", "validation": "mentalmanip_val.csv"})

def format_example(example):
    dialogue = example["dialogue"]
    technique = example["technique"] if example["technique"] else "None"
    return {"text": f"Dialogue: {dialogue}\nLabel: {technique}"}

dataset = dataset.map(format_example, remove_columns=dataset["train"].column_names)
dataset = dataset.map(lambda x: tokenizer(x["text"], truncation=True, padding="max_length", max_length=512), batched=True)

# Step 5: Fine-tune using Trainer
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    learning_rate=2e-4,
    num_train_epochs=3,
    fp16=True,
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()


In [ ]:

# Step 6: Evaluate the model on the validation set
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)


In [ ]:

# Step 7: Inference example
def predict(dialogue):
    prompt = f"Dialogue: {dialogue}\nLabel:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=20)
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated

# Example inference
example = "I did everything I could, but no one appreciates it."
print("Prediction:", predict(example))
